# Create metrics_config.toml

In this notebook we create the metrics_config.toml file. This file contains the SQL queries we want to do on the Delft-FIAT output to extract the data needed for the infographics.

In [ ]:
from pathlib import Path
import pandas as pd
from os import makedirs
import tomli_w

In [ ]:
model_root = Path('c:/Repositories/DT-flood/FloodAdapt_database/Humber/')

scenario_name = 'empty_event_current_no_measures'

fiat_out_path = model_root / 'output' / 'scenarios' / scenario_name / 'Impacts' / ('Impacts_detailed_'+scenario_name+'.csv')

metrics_path = model_root / 'static' / 'templates' / 'infometrics' / 'metrics_config.toml'

In [ ]:
fiat_out = pd.read_csv(fiat_out_path)

In [ ]:
fiat_out

In [ ]:
print(fiat_out['Primary Object Type'].unique())
print(fiat_out['Secondary Object Type'].unique())

## Querie

A querie needs to consist of the following:
- `name`: ID of the querie
- `description`: explanation of what to calculate
- `select`: SQL command as str how to calculate quantity (e.g. COUNT, SUM)
- `filter`: SQL command as str which data to select (e.g. which column)
- `long_name`
- `show_in_metrics_table`

In [ ]:
querie_list = []

Automate metric generation more. Make widgets with dropdown menu storing values into parameters (one for building type, one for stat to be queried, threshold etc) and make function that takes these parameters in and generates the right SQL querie and block in toml file

In [ ]:
querie_single = {
    "name": "TotalDamageEvent",
    "description": "Total damage to buildings",
    "select": "SUM('Total Damage')",
    "filter": "",
    "long_name": "Total building damage",
    "show_in_metrics_table": 'True'
}


In [ ]:
querie_list.append(querie_single)
print(querie_list)

## Aggregation

A list of some aggregation to be used. Any entry should have a corresponding entry in the site.toml file under fiat.aggregation

In [ ]:
agg_list = []

## Write toml file

In [ ]:
metric_dict = {
    "queries": querie_list,
}

if len(agg_list) > 0:
    metric_dict.update({"aggregateBy": agg_list})
    
print(metric_dict)

In [ ]:
if not metrics_path.parent.exists():
    makedirs(metrics_path.parent)

with open(metrics_path,'wb') as f:
    tomli_w.dump(metric_dict,f)